In [1]:
import pandas as pd
import numpy as dragon
import pylab as p
import matplotlib.pyplot as plot
from collections import Counter
from statsmodels.tsa.arima_model import ARIMA
import re

#importing packages for the prediction of time-series data
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

from sklearn.metrics import mean_squared_error
modelList=list()
cropList=list()
marketList=list()

C:\Users\nishi\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
trainog=pd.read_excel('CropPricesComplete.xlsx')
crops=trainog['crop/commodity'].unique()
print(crops)
def test_stationarity(x):
    #Determing rolling statistics
    rolmean = x.rolling(window=22,center=False).mean()

    rolstd = x.rolling(window=12,center=False).std()
    
    #Plot rolling statistics:
    orig = plot.plot(x, color='blue',label='Original')
    mean = plot.plot(rolmean, color='red', label='Rolling Mean')
    std = plot.plot(rolstd, color='black', label = 'Rolling Std')
    plot.legend(loc='best')
    plot.title('Rolling Mean & Standard Deviation')
    plot.show(block=False)
    
    #Perform Dickey Fuller test    
    result=adfuller(x)
    print('ADF Stastistic: %f'%result[0])
    print('p-value: %f'%result[1])
    pvalue=result[1]
    for key,value in result[4].items():
         if result[0]>value:
            print("The graph is non stationery")
            break
         else:
            print("The graph is stationery")
            break;
    print('Critical values:')
    for key,value in result[4].items():
        print('\t%s: %.3f ' % (key, value))
        

['Rice' 'Sesamum' 'Soyabean' 'Safflower' 'Mustard' 'Arhar Dal' 'Bajra'
 'Jowar' 'Maize' 'Wheat']


In [5]:
from statsmodels.tsa.stattools import adfuller
for i in crops:  
    train=trainog
    train=train[(train['Minimum Price(Rs./Quintal)']!="NR")]
    train=train[(train['crop/commodity']==i)]
    market=train['Market'].unique()
    print("For crop "+i)
    markets=list()
    models=list()
    crops=list()
    print(market)
    for j in market:
        train=train[(train['Market']==j)]
#         print(train.size)
        if(train.size>90):
            markets.append(j)
            data = train['Modal Price(Rs./Quintal)']
            Date1 = train['Date']
            train1 = train[['Date','Modal Price(Rs./Quintal)']]
            train2 = train1.set_index('Date')
            train2.sort_index(inplace=True)
            ts = train2['Modal Price(Rs./Quintal)']      
            test_stationarity(ts)
            ts_list=ts.tolist()
            ts_log = dragon.log(ts_list)
            # ts.apply(dragon.log)
            plot.plot(ts_log,color="green")
            plot.show()
            # print(type(ts_log))
            test_stationarity(pd.Series(ts_log))
            ts_log_diff = ts_log - pd.Series(ts_log).shift()
            ts_log_diff.dropna(inplace=True)
            test_stationarity(ts_log_diff)
            model = ARIMA(ts_log, order=(1,1,0))  
            results_ARIMA = model.fit(disp=-1)  
            plot.plot(ts_log_diff)
            plot.plot(results_ARIMA.fittedvalues, color='red')
            plot.title('RSS: %.7f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))
            plot.show()
            model = ARIMA(ts_log, order=(0,1,1))  
            results_MA = model.fit(disp=-1)  
            plot.plot(ts_log_diff)
            plot.plot(results_MA.fittedvalues, color='red')
            plot.title('RSS: %.7f'% sum((results_MA.fittedvalues-ts_log_diff)**2))
            plot.show()
            from statsmodels.tsa.arima_model import ARIMA
            model = ARIMA(ts_log, order=(2,1,0))
            results_ARIMA = model.fit(disp=-1)
            print (str(results_ARIMA)+" "+i+" "+j)
            models.append(results_ARIMA)
            crops.append(i)
            markets.append(j)
        else :
            print("No data")   
    modelList.append(models)
    marketList.append(markets)
    cropList.append(crops)

In [9]:
print(marketList)
# print(modelList)
print(cropList)

[['Alibagh', 'Alibagh'], [], ['Aarni', 'Aarni'], ['Ausa', 'Ausa'], ['Jalana', 'Jalana'], ['Kolhapur(Laxmipuri)', 'Kolhapur(Laxmipuri)'], [], ['Aatpadi', 'Aatpadi'], [], ['Achalpur', 'Achalpur'], ['Alibagh', 'Alibagh'], [], ['Aarni', 'Aarni'], ['Ausa', 'Ausa'], ['Jalana', 'Jalana'], ['Kolhapur(Laxmipuri)', 'Kolhapur(Laxmipuri)'], [], ['Aatpadi', 'Aatpadi'], [], ['Achalpur', 'Achalpur']]
['Rice', 'Soyabean', 'Safflower', 'Mustard', 'Arhar Dal', 'Jowar', 'Wheat', ['Rice'], [], ['Soyabean'], ['Safflower'], ['Mustard'], ['Arhar Dal'], [], ['Jowar'], [], ['Wheat']]
